In [187]:
import numpy as np
import importlib
import Layout
import Greedy

import copy 

#Reset libreria
importlib.reload(px)
importlib.reload(Greedy)

def showtime2(path):
    layout=px.read(path)
    #print ("Estado inicial: ")
    #for s in layout.stacks:
    #    print(s)
    #print()
    seleccionada = SSRC(layout)
 
    while seleccionada != -1:
        #print("SELECCIONADA:", seleccionada)
        
        #Se calcula cuantos contenedores deben retirarse
        columna=copy.deepcopy(layout.stacks[seleccionada])
        nb_unsorted_elements = px.remove_until(columna)

        #Se calculan los ranks
        rank = Greedy.get_ranks(layout.stacks[seleccionada])
        #print(rank)

        #Se retiran los elementos de la columna
        SDpp(layout, seleccionada, rank, nb_unsorted_elements)
        #for s in layout.stacks:
        #    print(s)
        #print("\n")

        #Se quitan los contenedores que se mantuvieron en su lugar para que no tire error la siguiente funcion
        for item in layout.stacks[seleccionada]:
            rank.pop(item, None)

        #print(nb_unsorted_elements)
        SFpp(layout,seleccionada,rank, nb_unsorted_elements)
        #for s in layout.stacks:
        #    print(s)
        #print("\n")

        #Se vuelve a seleccionar una columna
        seleccionada = SSRC(layout)

    #print("SELECCIONADA:", seleccionada)
    #print(len(layout.moves),"\t",layout.unsorted_stacks)
    return len(layout.moves)
    #print()

In [188]:
#SELECT SEMI RANDOM COLUMN
def SSRC(layout):
    
    size=len(layout.stacks)
    best=-1000000000
    select=-1
    
    posibles = []

    for i in range (0,size):

        aux = layout.is_sorted_stack(i)

        if aux == False:
            posibles.append(i)

    #Me di cuenta que con el que mayor problemas tiene tenemos menos problemas 
    #(necesita menos espacio al contrario del que tenga menos problemas)

    #Si no hay posibles se retorna -1 
    if len(posibles) == 0:
        return -1    
    
    #En casoq ue si hay posibles
    
    for i in posibles:
        aux = px.comparacion(layout.stacks[i])
        if aux > best:
            best = aux
            select = i 
    return select   

In [189]:
def SDpp(layout, s_o, rank, n_elements=100):
    capac = Greedy.capacity(layout,s_o) # espacio libre
    
    ss_o = layout.stacks[s_o]
    
    fixed_elements = np.maximum(len(ss_o)-n_elements,0)
    
    while len(layout.stacks[s_o]) > fixed_elements:
        top = Layout.gvalue(ss_o)
        slack = capac-len(ss_o) # holgura
        #print(rank[top])
        s_d = Layout.select_destination_stack (layout, s_o, max_pos=rank[top]+slack)
        pos = layout.H - len(layout.stacks[s_d])
        if rank[top] < pos - slack:  # rellenar stack s_d
            #c=layout.stacks[s_o].pop(-1)
            Greedy.fill_stack(layout, s_d, (pos - slack) - rank[top], s_o, rank)
            #layout.stacks[s_o].append(c)

        capac -= 1
        layout.move(s_o,s_d)

In [190]:
import sys
def SFpp(layout, s_d, rank, n):
    ub = 100
    cont = 0
    for r in rank:
        while True:
            (s, pos) = Greedy.search_highest(layout, r, ub, s_d)  
            #print("pos:",s,pos)
            #print("r,ub:",r,ub,layout.stacks[s])
            if layout.sorted_elements[s] > len(layout.stacks[s])+pos: break #element is sorted
            
            
            c=Greedy.force_move(layout, s, pos, s_d)
            
            if c == None:
                ub=layout.stacks[s][pos]-1
                if ub<r: 
                    for stack in layout.stacks:
                        print(stack)
                    print(str(ub+1)+ " del stack " +str(s)+ " no se puede volver a colocar en "+str(s_d)+"!!");
                    sys.exit()
            else: 
                ub=r
                cont+=1
                if cont == n: return
                if r==c: break
    while cont < n:
        Greedy.SF_move_d(layout, s_d)
        cont += 1

In [200]:
import Greedy as gr
import importlib
importlib.reload(Greedy)
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("Instancias") if not isfile(join("Instancias", f))]


for f in onlyfiles:
    if f == 'BF' or f == '3-3' or f== '3-x' or f=='5-4' or f=='5-5': continue
    #print(f)
    nmoves=0
    for i in range(1,41):
        filename = "Instancias/"+ f+ "/data"+f+"-"+str(i)+".dat"
        nmoves += showtime2(filename)    
    print (f,nmoves/ 40.0);

10-10 200.925
10-6 136.675
3-4 13.6
3-5 15.9
3-6 16.675
3-7 19.1
3-8 20.425
4-4 25.25
4-5 29.7
4-6 32.525
4-7 34.875
5-10 67.55
5-6 48.35
5-7 54.0
5-8 60.075
5-9 67.0
6-10 95.525
6-6 65.525


![image](https://i.imgur.com/XsBW9YR.png)